In [ ]:
import os
import pandas as pd
import param
import panel as pn
import numpy as np


import holoviews as hv
from holoviews import opts

from inter_view.dashboards import SegmentationDashBoard, ScatterDashBoard, LinkedScatterImageDashBoard
from improc.io import parse_collection, DCAccessor

pn.extension()
hv.extension('bokeh', width=100)

# config

- set the `basedir` as the folder where the aggregated features and parsed data collection are exported
- add entries in `channel_config` that match your  channels and labels as needed
- once you find reasonable intensity clipping bounds for each channel, you can specify the default here. If none are specifed, the bounds are adjusted automatically for each image until the slider is moved.
- points size, alpha, etc. of the scatter plot can also be adjusted here

In [ ]:
basedir='../../data/2D/props'

channel_config = {'TIF-OVR-MIP-1':{'cmap':'gray'},#, 'intensity_bounds':(300,10000), 'slider_limits':(0,20000)},
                  'TIF-OVR-MIP-2':{'cmap':'cyan'},#, 'intensity_bounds':(200,3000), 'slider_limits':(0,20000)},
                  'TIF-OVR-MIP-3':{'cmap':'yellow'},#, 'intensity_bounds':(100,1200), 'slider_limits':(0,20000)},
                  'TIF-OVR-MIP-4':{'cmap':'magenta'},#, 'intensity_bounds':(100,5000), 'slider_limits':(0,20000)},
                  'MASK':{'cmap':'glasbey_hv_16bit', 'raster_aggregator':'first', 'intensity_bounds':(0,2**16-1), 'bitdepth':16, 'opacity':0.2},
                  'CENTER-SURROUND':{'cmap':'glasbey_hv_16bit', 'raster_aggregator':'first', 'intensity_bounds':(0,2**16-1), 'bitdepth':16, 'opacity':0.2},
                  'SPX':{'cmap':'glasbey_hv_16bit', 'raster_aggregator':'first', 'intensity_bounds':(0,2**16-1), 'bitdepth':16, 'opacity':0.2},
                  'SKELETON':{'cmap':'glasbey_hv_16bit', 'raster_aggregator':'max', 'intensity_bounds':(0,2**16-1), 'bitdepth':16, 'opacity':1},}

opts.defaults(opts.Image('channel', frame_width=1000))
opts.defaults(opts.Image('channel.MASK', clipping_colors={'min': (0, 0, 0, 0)}, clim=(1,2**16-1)))
opts.defaults(opts.Image('channel.CENTER-SURROUND', clipping_colors={'min': (0, 0, 0, 0)}, clim=(1,2**16-1)))
opts.defaults(opts.Image('channel.SPX', clipping_colors={'min': (0, 0, 0, 0)}, clim=(1,2**16-1)))
opts.defaults(opts.Image('channel.SKELETON', clipping_colors={'min': (0, 0, 0, 0), 'max': 'red'}, clim=(0.4,0.6)))

opts.defaults(opts.Points('props_scatter', frame_width=1000, frame_height=1000, fontsize={'legend': 6}, size=10, alpha=1., line_color=None,
                          nonselection_fill_alpha=0.1, selection_fill_alpha=0.8, selection_line_color='black', selection_line_alpha=1.,))

# load file collection and features

In [ ]:
df = pd.read_hdf(os.path.join(basedir, 'output_collection.h5'), 'dc')

flat_props = pd.read_hdf(os.path.join(basedir, 'flat_props.h5'))

# combine channel and layer into a single index level
df = df[df.ext=='tif']
df.reset_index(['channel', 'zslice', 'field'], inplace=True)
df['overlay_layer'] = df.index.get_level_values('layer')

df.loc['TIF-OVR-MIP', 'overlay_layer'] = (df.loc['TIF-OVR-MIP', 'overlay_layer'] + '-' + df.loc['TIF-OVR-MIP', 'channel'].astype(str)).values
df = df.reset_index().set_index(['platedir', 'plate_row', 'plate_column', 'org_id', 'overlay_layer'])
df

# scatter dashboard

- the index levels of the feature dataframe `flat_props` will become interactive filters in the viewer

In [ ]:
from inter_view.dashboards import ScatterDashBoard

scat_db = ScatterDashBoard(df=flat_props.reset_index().set_index(['platedir', 'condition'], drop=False).sample(len(flat_props)),
                           x_key='MASK255_area',
                           y_key='MASK255_eccentricity',
                           color_key='MASK255_ch1_mean',
                           hover_keys=['condition', 'platedir', 'plate_row', 'plate_column', 'org_id'])

# scat_db.panel().servable()

# image dashboard

- the index levels of the collection dataframe `df` will become interactive drop down menu in the viewer
- if viewing overview images that contain an extra mask channel, a custom image loading function can be passed

In [ ]:
# from skimage.io import imread

# # custom file reading
# # reads only the first slice
# def read_first_slice(path):
#     return imread(path, img_num=0)

seg_db = SegmentationDashBoard(df=df,
                               multi_select_levels=['overlay_layer'],
                               channel_config=channel_config,
                               composite_channels=['TIF-OVR-MIP-1', 'TIF-OVR-MIP-2', 'TIF-OVR-MIP-3', 'TIF-OVR-MIP-4'],
                               overlay_channels=['CENTER-SURROUND', 'MASK', 'SKELETON', 'SPX'],
#                                loading_fun=read_first_slice,
                              )

# by default only display intensity channels
seg_db.file_widgets[4].value = ['TIF-OVR-MIP-1', 'TIF-OVR-MIP-2', 'TIF-OVR-MIP-3', 'TIF-OVR-MIP-4']

# seg_db.panel().servable()

# linked dashboard

- links the scatter plot to images bidirectionally
- features and data collection dataframes must have common columns for indexing (e.g. platedir, plate_row, plate_column, etc.)

In [ ]:
db = LinkedScatterImageDashBoard(seg_db=seg_db, scat_db=scat_db)

db.panel()